In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("image-segmentation", model="bricklerex/lane-detect-jds")

In [ ]:
# Load model directly
from transformers import AutoImageProcessor, SegformerForSemanticSegmentation

processor = AutoImageProcessor.from_pretrained("bricklerex/lane-detect-jds")
model = SegformerForSemanticSegmentation.from_pretrained("bricklerex/lane-detect-jds")

In [ ]:
from moviepy.editor import VideoFileClip
from PIL import Image
import numpy as np
import torch

def process_frame(frame):
    img = Image.fromarray(frame).resize((512, 512))
    inputs = processor(images=img, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    mask = outputs.logits.argmax(dim=1)[0].cpu().numpy()

    colored_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
    colored_mask[mask == 1] = [0, 255, 0]

    # Resize the colored_mask to match the size of the resized image
    colored_mask_resized = Image.fromarray(colored_mask).resize((512, 512))
    colored_mask_resized = np.array(colored_mask_resized)

    overlay = (0.6 * np.array(img) + 0.4 * colored_mask_resized).astype(np.uint8)
    return overlay

clip = VideoFileClip("lanes_clip.mp4")
processed_clip = clip.fl_image(process_frame)
processed_clip.write_videofile("lane_output.mp4", audio=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls /content/drive/MyDrive/Colab\ Notebooks

In [ ]:
import nbformat

# Load the current notebook
notebook_path = "/content/drive/MyDrive/Colab\ Notebooks\transferlanedetection"  # replace with your actual notebook name
with open(notebook_path, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

# Remove broken widgets metadata if it exists
if "widgets" in nb["metadata"]:
    del nb["metadata"]["widgets"]

# Save the cleaned notebook
with open(notebook_path, "w", encoding="utf-8") as f:
    nbformat.write(nb, f)

print("✅ Notebook metadata cleaned.")
